In this practice code, we are going to use Ozone dataset. Dictionary of this dataset can be found in https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/airquality.html

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf

In [2]:
url = "https://raw.githubusercontent.com/ga-students/DS-SF-24/master/Data/ozone.csv"
OzoneData = pd.read_csv(url)
OzoneData.describe()

#### Explore the dataset and decide which variables suffer from missing data

In [7]:
OzoneData.head()

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
4,NaN,NaN,14.3,56,5,5


Answer: Ozone & Solar.R

#### Let's drop rows that have missing values in all the columns you indentified above

Hint: in dropna() if you set how = 'all', it will only drop columns that are suffering from missing values at all varaibles you introduce in subset. If you want to get rid of the row that contains missing values in any of the variables you specify, then you shall set how = 'any'

df.dropna(how = 'all',subset = ['Var1','Var2','Var3'],inplace = True)

The above code will check if all 3 variables specified in df have missing values, if they all have missing values it will drop that row.

In [8]:
OzoneData.dropna(how = 'all', subset = ['Ozone', 'Solar.R'], inplace = True)

In [12]:
len(OzoneData)

151

#### Spoiler! If everything is going according to plan you shall be left by 151 observations. Also, it seemed like the first two varibales had missing values. Now please make a copy of your dataframe into a dataframe named OzoneImputeMean. Also, please use mean of the variables to fill in missing values in OzoneImputeMean

In [19]:
OzoneImputeMean = OzoneData
OzoneImputeMean.Ozone.fillna(value = np.mean(OzoneImputeMean.Ozone), inplace = True)
OzoneImputeMean['Solar.R'].fillna(value = np.mean(OzoneImputeMean['Solar.R']), inplace = True)

In [18]:
#Check if missing values are filled
OzoneImputeMean.head()

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.000000,7.4,67,5,1
1,36.0,118.000000,8.0,72,5,2
2,12.0,149.000000,12.6,74,5,3
3,18.0,313.000000,11.5,62,5,4
5,28.0,185.931507,14.9,66,5,6


#### Now it's time for imputing using linear regression lines

In [20]:
#### Before we start let's define dummy variables for variable Month - don't worry about day!

MonthDummy = pd.get_dummies(OzoneData.Month, prefix = 'Month')
del MonthDummy['Month_9']

OzoneData = pd.concat([OzoneData, MonthDummy], axis=1)

OzoneData.head()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
0,41.0,190.000000,7.4,67,5,1,1.0,0.0,0.0,0.0
1,36.0,118.000000,8.0,72,5,2,1.0,0.0,0.0,0.0
2,12.0,149.000000,12.6,74,5,3,1.0,0.0,0.0,0.0
3,18.0,313.000000,11.5,62,5,4,1.0,0.0,0.0,0.0
5,28.0,185.931507,14.9,66,5,6,1.0,0.0,0.0,0.0


In [21]:
#now let's explore correlation Matrix
OzoneData.corr()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
Ozone,1.000000,0.302970,-0.534163,0.630583,0.151089,-0.011472,-0.281111,-0.065820,0.251524,0.264054
Solar.R,0.302970,1.000000,-0.055581,0.271990,-0.073886,-0.147120,-0.023842,0.023896,0.175960,-0.073214
Wind,-0.534163,-0.055581,1.000000,-0.466032,-0.175317,0.042365,0.237781,0.046054,-0.144638,-0.166105
Temp,0.630583,0.271990,-0.466032,1.000000,0.397427,-0.136876,-0.637816,0.050753,0.318103,0.321680
Month,0.151089,-0.073886,-0.175317,0.397427,1.000000,-0.007727,-0.702257,-0.362131,-0.007201,0.355246
Day,-0.011472,-0.147120,0.042365,-0.136876,-0.007727,1.000000,0.011003,-0.017044,0.011471,0.011471
Month_5,-0.281111,-0.023842,0.237781,-0.637816,-0.702257,0.011003,1.000000,-0.242766,-0.247805,-0.247805
Month_6,-0.065820,0.023896,0.046054,0.050753,-0.362131,-0.017044,-0.242766,1.000000,-0.253080,-0.253080
Month_7,0.251524,0.175960,-0.144638,0.318103,-0.007201,0.011471,-0.247805,-0.253080,1.000000,-0.258333
Month_8,0.264054,-0.073214,-0.166105,0.321680,0.355246,0.011471,-0.247805,-0.253080,-0.258333,1.000000


#### What seems to be the list of best variables can define Ozone? how about Solar.R?

Answer: Ozone => Temp, Solar.R => Ozone 

#### Now let's use a regression model to predict Ozone. First drop NaN values in Ozone and save it in OzoneDroppedValues_Ozone. Then run a regression line on variables of interest and check significancy of your model if in a multi class dummy variable case, you see only a few of the dummy variables are not significant but the majority are,  you shall either drop all or keep all. Otherwise, selecting the base dummy will become important use these variables ['Solar.R','Wind','Temp','Month_5','Month_6','Month_7','Month_8'] to predict.


In [33]:
OzoneDroppedValues_Ozone = OzoneData.dropna(subset = ['Ozone'])
X1 = OzoneDroppedValues_Ozone[['Solar.R', 'Wind', 'Temp', 'Month_5', 'Month_6', 'Month_7', 'Month_8']]
y1 = OzoneDroppedValues_Ozone['Ozone']
lm1 = smf.ols(formula = 'y1 ~ X1', data = OzoneDroppedValues_Ozone).fit()
print(lm1.pvalues)

Intercept    0.0
X1[0]        0.0
X1[1]        0.0
X1[2]        0.0
X1[3]        0.0
X1[4]        0.0
X1[5]        0.0
X1[6]        0.0
dtype: float64


In [34]:
# now fill in null values of OzoneData['Ozone'] by predicted values
lm = LinearRegression()

lm.fit(X1, y1)

X = OzoneDroppedValues_Ozone[['Solar.R', 'Wind', 'Temp', 'Month_5', 'Month_6', 'Month_7', 'Month_8']]

OzoneData['Ozone'] = lm.predict(X)
OzoneData['Ozone'].fillna(value = OzoneData['Ozone'], inplace = True)
OzoneData.head()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
0,37.902359,190.000000,7.4,67,5,1,1.0,0.0,0.0,0.0
1,39.985348,118.000000,8.0,72,5,2,1.0,0.0,0.0,0.0
2,32.795274,149.000000,12.6,74,5,3,1.0,0.0,0.0,0.0
3,26.495151,313.000000,11.5,62,5,4,1.0,0.0,0.0,0.0
5,17.295052,185.931507,14.9,66,5,6,1.0,0.0,0.0,0.0


In [25]:
# Now repeat previous steps for Solar.R variable using ['Ozone','Wind','Temp]
OzoneDroppedValues_Solar = OzoneData.dropna(subset = ['Solar.R'])
X1 = OzoneDroppedValues_Solar[['Ozone', 'Wind', 'Temp']]
y1 = OzoneDroppedValues_Solar['Solar.R']
lm1 = smf.ols(formula = 'y1 ~ X1', data = OzoneDroppedValues_Solar).fit()
print(lm1.pvalues)

Intercept    0.740845
X1[0]        0.007049
X1[1]        0.055154
X1[2]        0.090145
dtype: float64


In [36]:
# now fill in null values of OzoneData['Solar.R'] by predicted values
X2 = OzoneDroppedValues_Solar[['Ozone']]
y2 = OzoneDroppedValues_Solar['Solar.R']
lm2 = smf.ols(formula = 'y2 ~ X2', data = OzoneDroppedValues_Solar).fit()
print(lm2.pvalues)

lm = LinearRegression()

lm.fit(X2, y2)

X = OzoneDroppedValues_Solar[['Ozone']]

OzoneData['Solar.R'] = lm.predict(X)
OzoneData['Solar.R'].fillna(value = OzoneData['Solar.R'], inplace = True)
OzoneData.head()

Intercept    1.073069e-23
X2           1.560239e-04
dtype: float64


,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
0,37.902359,184.882648,7.4,67,5,1,1.0,0.0,0.0,0.0
1,39.985348,180.238844,8.0,72,5,2,1.0,0.0,0.0,0.0
2,32.795274,157.948585,12.6,74,5,3,1.0,0.0,0.0,0.0
3,26.495151,163.521149,11.5,62,5,4,1.0,0.0,0.0,0.0
5,17.295052,172.808757,14.9,66,5,6,1.0,0.0,0.0,0.0


#### Now check your filled data - if your predicted values are more than maximum or less than minimum, replace them by max and min

In [37]:
linreg = LinearRegression()

X = OzoneDroppedValues_Ozone[['Solar.R', 'Wind', 'Temp', 'Month_5', 'Month_6', 'Month_7', 'Month_8']]
y = OzoneDroppedValues_Ozone['Ozone']

linreg.fit(X,y)

y_hat = linreg.predict(X)

Standard_error = ( sum( (y - y_hat) ** 2 )/(len(y) - 2) ) ** .5

print(Standard_error)

y_min = y.min()
y_max = y.max()

X = OzoneData[['Solar.R', 'Wind', 'Temp']]

OzoneData['predicted_ozone_error'] = linreg.predict(X) + np.random.normal(0, Standard_error, len(OzoneData))

print('Minimum Values before adjustment %f:'  %OzoneData['predicted_ozone_error'].min())

##### Here we make sure we don't predict beyond or below our age limits

OzoneData.loc[(OzoneData['predicted_ozone_error'] < y_min), 'predicted_ozone_error'] = y_min 
OzoneData.loc[(OzoneData['predicted_ozone_error'] > y_max), 'predicted_ozone_error'] = y_max

print('Minimum Values after adjustment %f:'  %OzoneData['predicted_ozone_error'].min())


OzoneData['Ozone'].fillna(value = OzoneData['predicted_ozone_error'], inplace = True)

OzoneData.head(20)

6.31651493716e-14


ValueError: shapes (151,3) and (7,) not aligned: 3 (dim 1) != 7 (dim 0)

#### Bonus: Repeat the above procedure, this time fill in missing values using regression with errors. 

In [ ]:
url = "https://raw.githubusercontent.com/ga-students/DS-SF-24/master/Data/ozone.csv"
OzoneData = pd.read_csv(url)